In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_hf.txt"
testDataPath = "s3://corpus-2/dataset/corpus_5_new.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        test_data = [[] for i in range(num)]
        
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])
                
        with open(testDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                test_data[index].append([content,label])

        for i in range(num):
            # np.random.shuffle(train_data[i])
            train_data[i], test_data[i] = shuffle(train_data[i], test_data[i])
            train_data[i] = np.asarray(train_data[i])
            test_data[i] = np.asarray(test_data[i])

        train_data, test_data = shuffle(train_data, test_data)
        return train_data, test_data
    
    
    train_data = []
    test_data = []
    
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
            
    with open(testDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            test_data.append([content,label])
            
    train_data, test_data = shuffle(train_data, test_data)
    return np.asarray(train_data), np.asarray(test_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(train_data, test_data, split_type):
    
    print(split_type)
    
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)
        
        te_x = []
        te_y = []
        for ti in test_data:
            x_test, y_test = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            te_x.append(x_test)
            te_y.append(y_test)

        te_x = np.asarray(te_x)
        te_y = np.asarray(te_y)
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(te_x[test_i],te_y[test_i])
            test_acc.append(classifier.train(
                X_train=train_x,
                y_train=train_y,
                X_eval=test_x,
                y_eval=test_y,
                categories=categories,
                epochs=50
            ))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        te_x, te_y = process_file(test_data[:,0], test_data[:,1], word_to_id, num_classes, seq_length)
        # print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            test_acc.append(classifier.train(
                X_train=tx[train_i],
                y_train=ty[train_i],
                X_eval=te_x[test_i],
                y_eval=te_y[test_i],
                categories=categories,
                epochs=50
            ))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)  # label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [5]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)
 
seq_length = 41  # seq_length = 37  TREC

Loading GloVe!
Completed!
400000 100 400000


In [6]:
class Classifier:

    def __init__(self, model, input_length, output_length):
        self.model = model
        self.input_length = input_length
        self.output_length = output_length

    def compile(self, batch_size=32):
        self._ds_x = tf.placeholder(tf.float32, [None, self.input_length])
        self._ds_y = tf.placeholder(tf.float32, [None, self.output_length])

        ds = tf.data.Dataset.from_tensor_slices((self._ds_x, self._ds_y))
        ds = ds.batch(batch_size)

        self._ds_it = ds.make_initializable_iterator()
        self._input, self._labels = self._ds_it.get_next()

        self._features = self.model(self._input)
        self._output = _create_dense_layer(self._features, self.output_length)

        self._create_acc_computations()
        self._create_backpropagation()

    def _create_acc_computations(self):
        self._predictions = tf.argmax(self._output, 1)
        labels = tf.argmax(self._labels, 1)
        self._accuracy = tf.reduce_mean(
            tf.cast(tf.equal(self._predictions, labels), 'float32'))

    def _create_backpropagation(self):
        losses = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self._output,
            labels=self._labels)
        self._loss = tf.reduce_mean(losses)

        optimizer = tf.train.AdamOptimizer(0.001)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        grads_and_vars = optimizer.compute_gradients(self._loss)

        self._train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=global_step)

    def summary(self):
        print('input:', self._input.shape)
        self.model.summary()
        print('output:', self._output.shape)

    def train(self, X_train, y_train, X_eval, y_eval, categories, epochs=20, require_improve=3):
        
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())
        
        best_vac_acc = 0.0
        last_improved = 0
        
        for e in range(epochs):
            start_time = time.time()
            loss, acc = self._train(X_train, y_train, session)
            duration = time.time() - start_time

            val_loss, val_acc = self._eval(X_eval, y_eval, session)
            
            if val_acc > best_vac_acc:
                best_vac_acc = val_acc
                last_improved = e
                improved_str = '*'
            else:
                improved_str = ''
            
            output = 'Epoch: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
            print(output.format(e + 1, loss, acc, val_loss, val_acc, duration, improved_str))
            
            if e - last_improved > require_improve:
                print("No optimization for a long time, auto-stopping...")
                
                y_test_cls = np.argmax(y_eval, 1)  # 获得类别
                y_test_pred_cls = np.argmax(self.predict(X_eval, session), 1)
                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                
                # evaluate
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                print("Confusion Matrix...")
                print(metrics.confusion_matrix(y_test_cls, y_test_pred_cls))
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                break
        # endfor
        session.close()
        return accuracy_score

    def _train(self, X_train, y_train, session):
        import numpy as np

        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_train,
                self._ds_y: y_train
            })
        loss, acc, = [], []
        while True:
            try:
                _, vloss, vacc = session.run(
                    fetches=[self._train_op, self._loss, self._accuracy])

                loss.append(vloss)
                acc.append(vacc)
            except tf.errors.OutOfRangeError:
                break
        # endwhile

        loss, acc = np.mean(loss), np.mean(acc)
        return loss, acc

    def _eval(self, X_val, y_val, session):
        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_val,
                self._ds_y: y_val
            })

        loss, acc, = 0, 0
        while True:
            try:
                l, vloss, vacc = session.run(
                    fetches=[self._labels, self._loss, self._accuracy])

                loss += vloss * len(l)
                acc += vacc * len(l)
            except tf.errors.OutOfRangeError:
                break

        return loss / len(X_val), acc / len(X_val)

    def predict(self, X, session):
        session.run(self._ds_it.initializer,
                         feed_dict={
                             self._ds_x: X,
                             self._ds_y: np.empty((len(X), self.output_length))
                         }
                         )

        pred = list()
        while True:
            try:
                ppred = session.run(tf.nn.softmax(self._output))

                pred.extend(map(lambda l: l.tolist(), ppred))
            except tf.errors.OutOfRangeError:
                break

        return pred

def _create_dense_layer(x, output_length):
    '''Creates a dense layer
    '''
    input_size = x.shape[1].value
    W = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[input_size, output_length],
            stddev=0.1))
    b = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[output_length]))

    dense = tf.nn.xw_plus_b(x, W, b)

    return dense

In [7]:
class KimConvolutionalModel:
    '''
    Implementation proposal of: https://arxiv.org/pdf/1408.5882.pdf
    '''
    def __init__(self,
        embeddings_configuration,
        conv_configurations = [(3, 100), (4, 100), (5, 100)],
        drop_rate           = 0.5):
        '''Constructor.
        # Parameters:
        embeddings: List of embeddings configuration. Each configuration is a
            pair of the form (embedding, trainable). `embedding` is a numpy
            array and `trainable` is a boolean that indicates whether that
            embedding is trainable or not.
        conv_configurations: List of pairs. Each pair represents a
            convolution configuration. Each configuration determines the
            size and number of each filter.
        '''

        self._embeddings_configuration = embeddings_configuration
        self._conv_configurations = conv_configurations
        self._drop_rate = drop_rate

    def __call__(self, input):
        self._embeddings_tf = tf.stack(
            values = [
                self._create_embedding_layer(e, input)
                for e in self._embeddings_configuration],
            axis = 1
        )

        self._convolutions_tf = self._create_convolutional_layers(
            self._conv_configurations, self._embeddings_tf)
        
        self._add_tf = self._create_add_layers(self._convolutions_tf)

        self._poolings_tf = self._create_maxpooling_layer(
            self._add_tf)

        self._reshape_tf = self._create_reshape_layer(self._poolings_tf)
        self._dropout_tf = tf.nn.dropout(
            self._reshape_tf,
            keep_prob = self._drop_rate)

        return self._dropout_tf

    def summary(self):
        print('embedding:', str(self._embeddings_tf.shape))
        for c in self._convolutions_tf:
            print('conv:', str(c.shape))
        for a in self._add_tf:
            print('add:', str(a.shape))
        for p in self._poolings_tf:
            print('pool:', str(p.shape))
        print('reshape:', str(self._reshape_tf.shape))

    def _create_embedding_layer(self, embedding_configuration, input_x):
        return tf.nn.embedding_lookup(
            params = tf.Variable(
                initial_value = embedding_configuration[0],
                trainable     = embedding_configuration[1]),
            ids = tf.cast(input_x, 'int32')
        )

    def _create_convolutional_layers(self, configuration, input_embedding):
        '''Creates the convolutional layers.
        # Parameters:
        configuration: A list. It must be of the form
            [(filter_size, num_filters), ...]
        # Returns:
        A list of tensorflow nodes. Each node 'i' computes the configuration 'i'.
        '''
        convolutions = []
        for filter_height, num_filters in configuration:
            filter_width = input_embedding.shape[3].value
            filter_shape = [1, filter_height, filter_width, num_filters]

            # Create weights and bias
            W = tf.Variable(
                initial_value=tf.truncated_normal(
                    shape=filter_shape,
                    stddev=0.1))
            b = tf.Variable(
                initial_value=tf.truncated_normal(
                    shape=[num_filters]))

            conv = tf.nn.conv2d(
                input=input_embedding,
                filter=W,
                strides=[1, 1, 1, 1],
                padding="VALID")
            bias = tf.nn.bias_add(conv, b)
            h = tf.nn.relu(bias)
            convolutions.append(h)

        return convolutions

    def _create_add_layers(self, convolutions):
        return [
            tf.reduce_sum(
                input_tensor = c,
                axis=1,
                keepdims=True)
            for c in convolutions
        ]

    def _create_maxpooling_layer(self, tensors):
        '''Creates the maxpooling layer. Computes maxpooling on each node
        # Parameters:
        input_convolutions: List of tensorflow nodes.
        # Returns:
        A list of tensorflow nodes. Each node 'i' computes the maxpooling of node 'i'
        '''
        return [
            tf.reshape(
                tensor = tf.nn.max_pool(
                    value=t,
                    ksize=[1, 1, t.shape[2], 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID'),
                shape = [-1, t.shape[3]]
            )
            for t in tensors
        ]

    def _create_reshape_layer(self, tensors):
        '''Creates a flatten layer
        '''
        return tf.concat(tensors, axis=1)

In [8]:
word_vector = embedding.astype('float32')
model = KimConvolutionalModel(
        embeddings_configuration=[
            (word_vector, True)
        ]
    )

classifier = Classifier(
    model=model,
    input_length=seq_length,
    output_length=num_classes)

classifier.compile(batch_size=32)
classifier.summary()

input: (?, 41)
embedding: (?, 1, 41, 100)
conv: (?, 1, 39, 100)
conv: (?, 1, 38, 100)
conv: (?, 1, 37, 100)
add: (?, 1, 39, 100)
add: (?, 1, 38, 100)
add: (?, 1, 37, 100)
pool: (?, 100)
pool: (?, 100)
pool: (?, 100)
reshape: (?, 300)
output: (?, 10)


In [ ]:
kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data, test_data = dataset_split(info)
    test_acc_split.append(train_split_data(train_data, test_data, split_type))

random
Fold:  1
Epoch: 1, Train Loss:  1.888, Train Acc: 42.99%, Val Loss:  1.098, Val Acc: 63.89%, Time: 7.95s *
Epoch: 2, Train Loss: 0.8674, Train Acc: 73.07%, Val Loss: 0.7922, Val Acc: 75.64%, Time: 3.51s *
Epoch: 3, Train Loss: 0.5928, Train Acc: 81.66%, Val Loss: 0.6611, Val Acc: 80.20%, Time: 3.50s *
Epoch: 4, Train Loss: 0.4193, Train Acc: 87.35%, Val Loss: 0.6013, Val Acc: 81.98%, Time: 3.49s *
Epoch: 5, Train Loss: 0.3121, Train Acc: 90.70%, Val Loss: 0.5026, Val Acc: 85.83%, Time: 3.48s *
Epoch: 6, Train Loss:  0.225, Train Acc: 93.42%, Val Loss: 0.5083, Val Acc: 84.97%, Time: 3.51s 
Epoch: 7, Train Loss: 0.1722, Train Acc: 94.82%, Val Loss: 0.4981, Val Acc: 86.18%, Time: 3.48s *
Epoch: 8, Train Loss: 0.1403, Train Acc: 95.74%, Val Loss: 0.4871, Val Acc: 86.18%, Time: 3.48s 
Epoch: 9, Train Loss: 0.1105, Train Acc: 96.73%, Val Loss: 0.5112, Val Acc: 86.82%, Time: 3.48s *
Epoch: 10, Train Loss: 0.09464, Train Acc: 97.38%, Val Loss: 0.4864, Val Acc: 87.61%, Time: 3.46s *
Epoc

Epoch: 10, Train Loss: 0.09183, Train Acc: 97.43%, Val Loss: 0.4838, Val Acc: 87.18%, Time: 3.47s 
Epoch: 11, Train Loss: 0.07675, Train Acc: 97.81%, Val Loss: 0.4851, Val Acc: 87.32%, Time: 3.49s 
Epoch: 12, Train Loss: 0.06849, Train Acc: 97.81%, Val Loss: 0.4836, Val Acc: 87.46%, Time: 3.51s 
Epoch: 13, Train Loss: 0.0603, Train Acc: 98.25%, Val Loss:  0.459, Val Acc: 88.11%, Time: 3.47s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.85      0.94      0.89       137
                   Filter       0.87      0.81      0.84       153
    Compute Derived Value       0.87      0.85      0.86       175
            Find Extremum       0.91      0.96      0.93       158
                     Sort       0.91      0.93      0.92       123
          Determine Range       0.85      0.88      0.86       120
Characterize Distribution       0.95      0.86     

Epoch: 1, Train Loss:  1.862, Train Acc: 44.35%, Val Loss:  1.123, Val Acc: 65.36%, Time: 3.64s *
Epoch: 2, Train Loss: 0.8502, Train Acc: 73.66%, Val Loss: 0.7952, Val Acc: 75.55%, Time: 3.45s *
Epoch: 3, Train Loss: 0.5747, Train Acc: 82.58%, Val Loss: 0.6723, Val Acc: 79.47%, Time: 3.46s *
Epoch: 4, Train Loss: 0.4174, Train Acc: 87.49%, Val Loss: 0.6243, Val Acc: 82.54%, Time: 3.44s *
Epoch: 5, Train Loss: 0.3017, Train Acc: 90.74%, Val Loss: 0.5697, Val Acc: 82.39%, Time: 3.56s 
Epoch: 6, Train Loss: 0.2353, Train Acc: 92.83%, Val Loss: 0.5151, Val Acc: 84.32%, Time: 3.45s *
Epoch: 7, Train Loss: 0.1769, Train Acc: 94.75%, Val Loss: 0.4602, Val Acc: 86.32%, Time: 3.46s *
Epoch: 8, Train Loss: 0.1415, Train Acc: 95.79%, Val Loss: 0.4764, Val Acc: 85.89%, Time: 3.45s 
Epoch: 9, Train Loss: 0.1161, Train Acc: 96.39%, Val Loss: 0.4777, Val Acc: 87.31%, Time: 3.47s *
Epoch: 10, Train Loss: 0.09798, Train Acc: 97.15%, Val Loss: 0.4616, Val Acc: 86.96%, Time: 3.45s 
Epoch: 11, Train Loss

Epoch: 11, Train Loss: 0.08792, Train Acc: 97.25%, Val Loss: 0.4183, Val Acc: 88.31%, Time: 3.54s *
Epoch: 12, Train Loss: 0.07505, Train Acc: 97.78%, Val Loss: 0.4707, Val Acc: 87.95%, Time: 3.46s 
Epoch: 13, Train Loss: 0.06346, Train Acc: 98.15%, Val Loss: 0.4574, Val Acc: 88.52%, Time: 3.45s *
Epoch: 14, Train Loss: 0.05576, Train Acc: 98.34%, Val Loss: 0.4465, Val Acc: 88.88%, Time: 3.45s *
Epoch: 15, Train Loss: 0.05196, Train Acc: 98.51%, Val Loss: 0.4456, Val Acc: 88.17%, Time: 3.45s 
Epoch: 16, Train Loss: 0.04728, Train Acc: 98.58%, Val Loss: 0.4443, Val Acc: 88.95%, Time: 3.43s *
Epoch: 17, Train Loss: 0.0382, Train Acc: 98.86%, Val Loss: 0.4716, Val Acc: 87.95%, Time: 3.49s 
Epoch: 18, Train Loss:  0.037, Train Acc: 98.92%, Val Loss: 0.4779, Val Acc: 88.31%, Time: 3.45s 
Epoch: 19, Train Loss: 0.03624, Train Acc: 98.90%, Val Loss: 0.4724, Val Acc: 88.95%, Time: 3.46s 
Epoch: 20, Train Loss: 0.03628, Train Acc: 98.94%, Val Loss: 0.4629, Val Acc: 89.31%, Time: 3.48s *
Epoch: 

Epoch: 3, Train Loss: 0.5839, Train Acc: 82.35%, Val Loss:  0.707, Val Acc: 77.98%, Time: 3.45s *
Epoch: 4, Train Loss: 0.4043, Train Acc: 88.00%, Val Loss: 0.6262, Val Acc: 81.90%, Time: 3.44s *
Epoch: 5, Train Loss:  0.292, Train Acc: 91.26%, Val Loss: 0.6106, Val Acc: 82.75%, Time: 3.45s *
Epoch: 6, Train Loss: 0.2284, Train Acc: 93.26%, Val Loss: 0.5735, Val Acc: 83.46%, Time: 3.47s *
Epoch: 7, Train Loss: 0.1769, Train Acc: 94.98%, Val Loss: 0.5769, Val Acc: 84.11%, Time: 3.44s *
Epoch: 8, Train Loss: 0.1323, Train Acc: 96.23%, Val Loss:  0.551, Val Acc: 84.03%, Time: 3.51s 
Epoch: 9, Train Loss: 0.1106, Train Acc: 96.66%, Val Loss: 0.5363, Val Acc: 86.10%, Time: 3.46s *
Epoch: 10, Train Loss: 0.09734, Train Acc: 97.27%, Val Loss: 0.5726, Val Acc: 84.53%, Time: 3.46s 
Epoch: 11, Train Loss: 0.07981, Train Acc: 97.55%, Val Loss: 0.5803, Val Acc: 85.32%, Time: 3.46s 
Epoch: 12, Train Loss: 0.06823, Train Acc: 97.97%, Val Loss: 0.5721, Val Acc: 87.03%, Time: 3.46s *
Epoch: 13, Train 

...

Epoch: 1, Train Loss:  2.472, Train Acc: 29.08%, Val Loss:  1.722, Val Acc: 42.82%, Time: 3.68s *
Epoch: 2, Train Loss:  1.257, Train Acc: 59.68%, Val Loss:  1.492, Val Acc: 49.43%, Time: 3.45s *
Epoch: 3, Train Loss: 0.9283, Train Acc: 71.52%, Val Loss:  1.358, Val Acc: 55.97%, Time: 3.49s *
Epoch: 4, Train Loss: 0.6988, Train Acc: 78.61%, Val Loss:  1.318, Val Acc: 59.67%, Time: 3.46s *
Epoch: 5, Train Loss: 0.5356, Train Acc: 83.68%, Val Loss:  1.345, Val Acc: 61.38%, Time: 3.46s *
Epoch: 6, Train Loss: 0.4219, Train Acc: 87.31%, Val Loss:  1.283, Val Acc: 61.95%, Time: 3.46s *
Epoch: 7, Train Loss: 0.3214, Train Acc: 90.70%, Val Loss:  1.379, Val Acc: 62.52%, Time: 3.47s *
Epoch: 8, Train Loss: 0.2559, Train Acc: 92.46%, Val Loss:  1.354, Val Acc: 64.30%, Time: 3.46s *
Epoch: 9, Train Loss: 0.2104, Train Acc: 93.86%, Val Loss:  1.515, Val Acc: 62.66%, Time: 3.45s 
Epoch: 10, Train Loss: 0.1663, Train Acc: 95.44%, Val Loss:  1.442, Val Acc: 64.44%, Time: 3.45s *
Epoch: 11, Train Los

Epoch: 1, Train Loss:  2.555, Train Acc: 27.76%, Val Loss:  1.575, Val Acc: 48.16%, Time: 3.66s *
Epoch: 2, Train Loss:  1.252, Train Acc: 59.85%, Val Loss:   1.34, Val Acc: 57.21%, Time: 3.55s *
Epoch: 3, Train Loss: 0.9138, Train Acc: 71.25%, Val Loss:  1.217, Val Acc: 62.16%, Time: 3.46s *
Epoch: 4, Train Loss: 0.6873, Train Acc: 78.70%, Val Loss:  1.196, Val Acc: 64.57%, Time: 3.44s *
Epoch: 5, Train Loss: 0.5311, Train Acc: 83.94%, Val Loss:  1.196, Val Acc: 66.05%, Time: 3.46s *
Epoch: 6, Train Loss: 0.4111, Train Acc: 87.63%, Val Loss:   1.26, Val Acc: 65.84%, Time: 3.45s 
Epoch: 7, Train Loss: 0.3218, Train Acc: 90.28%, Val Loss:  1.272, Val Acc: 65.77%, Time: 3.47s 
Epoch: 8, Train Loss: 0.2512, Train Acc: 92.50%, Val Loss:   1.39, Val Acc: 63.86%, Time: 3.45s 
Epoch: 9, Train Loss: 0.2004, Train Acc: 94.22%, Val Loss:  1.433, Val Acc: 65.91%, Time: 3.44s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision  

table
Fold:  1
Epoch: 1, Train Loss:  2.094, Train Acc: 39.68%, Val Loss:   1.58, Val Acc: 51.14%, Time: 3.84s *
Epoch: 2, Train Loss: 0.9821, Train Acc: 69.70%, Val Loss:  1.235, Val Acc: 62.52%, Time: 3.67s *
Epoch: 3, Train Loss: 0.6658, Train Acc: 79.52%, Val Loss:  1.066, Val Acc: 69.08%, Time: 3.64s *
Epoch: 4, Train Loss: 0.4926, Train Acc: 84.92%, Val Loss:    1.1, Val Acc: 69.34%, Time: 3.65s *
Epoch: 5, Train Loss: 0.3671, Train Acc: 89.12%, Val Loss:  1.143, Val Acc: 68.94%, Time: 3.65s 
Epoch: 6, Train Loss: 0.2839, Train Acc: 91.67%, Val Loss:  1.133, Val Acc: 70.55%, Time: 3.63s *
Epoch: 7, Train Loss: 0.2171, Train Acc: 93.41%, Val Loss:  1.073, Val Acc: 72.16%, Time: 3.74s *
Epoch: 8, Train Loss: 0.1778, Train Acc: 94.47%, Val Loss:  1.246, Val Acc: 70.95%, Time: 3.64s 
Epoch: 9, Train Loss: 0.1424, Train Acc: 95.64%, Val Loss:  1.199, Val Acc: 71.75%, Time: 3.64s 
Epoch: 10, Train Loss: 0.1146, Train Acc: 96.78%, Val Loss:  1.368, Val Acc: 67.20%, Time: 3.66s 
Epoch: 1

Epoch: 3, Train Loss: 0.7198, Train Acc: 77.91%, Val Loss:  1.226, Val Acc: 64.06%, Time: 3.24s *
Epoch: 4, Train Loss: 0.5231, Train Acc: 84.16%, Val Loss:  1.169, Val Acc: 66.93%, Time: 3.27s *
Epoch: 5, Train Loss: 0.3872, Train Acc: 88.29%, Val Loss:  1.243, Val Acc: 66.93%, Time: 3.25s *
Epoch: 6, Train Loss: 0.2966, Train Acc: 91.36%, Val Loss:  1.227, Val Acc: 67.35%, Time: 3.25s *
Epoch: 7, Train Loss: 0.2326, Train Acc: 93.10%, Val Loss:  1.232, Val Acc: 68.59%, Time: 3.27s *
Epoch: 8, Train Loss: 0.1875, Train Acc: 94.67%, Val Loss:   1.35, Val Acc: 66.88%, Time: 3.23s 
Epoch: 9, Train Loss: 0.1463, Train Acc: 95.79%, Val Loss:  1.443, Val Acc: 67.67%, Time: 3.30s 
Epoch: 10, Train Loss: 0.1208, Train Acc: 96.43%, Val Loss:  1.437, Val Acc: 68.83%, Time: 3.26s *
Epoch: 11, Train Loss: 0.1004, Train Acc: 97.17%, Val Loss:  1.585, Val Acc: 66.42%, Time: 3.25s 
Epoch: 12, Train Loss: 0.07986, Train Acc: 97.69%, Val Loss:  1.652, Val Acc: 67.07%, Time: 3.25s 
Epoch: 13, Train Los

Epoch: 6, Train Loss:  0.279, Train Acc: 92.01%, Val Loss:  1.378, Val Acc: 66.74%, Time: 3.11s *
Epoch: 7, Train Loss:  0.213, Train Acc: 93.75%, Val Loss:  1.541, Val Acc: 65.57%, Time: 3.11s 
Epoch: 8, Train Loss: 0.1742, Train Acc: 94.84%, Val Loss:  1.616, Val Acc: 65.27%, Time: 3.10s 
Epoch: 9, Train Loss: 0.1375, Train Acc: 96.08%, Val Loss:  1.622, Val Acc: 65.35%, Time: 3.12s 
Epoch: 10, Train Loss: 0.1075, Train Acc: 96.89%, Val Loss:   1.66, Val Acc: 65.87%, Time: 3.12s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.81      0.40      0.54       281
                   Filter       0.59      0.60      0.60       235
    Compute Derived Value       0.63      0.54      0.58       237
            Find Extremum       0.74      0.76      0.75       259
                     Sort       0.85      0.78      0.81       267
          Determine Range 